In [ ]:
#qcow22mount
#https://www.jamescoyle.net/how-to/1818-access-a-qcow2-virtual-disk-image-from-the-host
sudo modprobe nbd
sudo qemu-nbd -c /dev/nbd0 ~/Documents/ESP.qcow2
sudo partx -l /dev/nbd0
sudo mkdir /mnt/ESP
sudo mount /dev/nbd0p1 /mnt/ESP
#sudo cp -rf /mnt/ESP/* /media/user/SED/VBOX/ESP/

#CLOVER INSTALL
git clone https://github.com/m13253/clover-linux-installer.git
cd clover*
./install.sh
#/dev/nvme1n1
#/dev/nvme1n1p1
#/dev/sdc
#/dev/sdc1

#mountESP
#W10
disk="nvme0n1p1"

#MacOS
disk="sdc1"
#PopOS
disk="nvme1n1p1"
sudo umount /mnt/ESP0
sudo mkdir /mnt/ESP0
sudo mount /dev/$disk /mnt/ESP0
sudo nautilus /mnt/ESP0

#copy apfs.efi to ESP
#cd /media/user/SED/VBOX/ESP/EFI/CLOVER
#sudo cp -r ./drivers64UEFI /mnt/MOEFI/EFI/CLOVER/drivers64UEFI
sudo rm -rf /mnt/ESP0/*
sudo cp -rf /media/user/SED/VBOX/ESP/* /mnt/ESP0/
sudo umount /mnt/ESP0/

#/boot/efi/EFI/CLOVER
#sudo ls /boot/efi/EFI/CLOVER
#sudo rm -rf /boot/efi/EFI/CLOVER
sudo cp -rf /media/user/SED/VBOX/ESP/* /boot/efi/

In [ ]:
#VENTOY
#https://ostechnix.com/how-to-create-multiboot-usb-drives-with-ventoy-in-linux/
wget https://github.com/ventoy/Ventoy/releases/download/v1.0.17/ventoy-1.0.17-linux.tar.gz
tar -xvjf ventoy-1.0.17-linux.tar.gz
cd ventoy*
sudo fdisk -l
#sudo sh Ventoy2Disk.sh -I /dev/sdc
sudo sh Ventoy2Disk.sh -I -s /dev/sdc
#Alternatively, go to the location where you saved ISO files and copy all ISO files from command line with rsync like below:
#rsync *.iso /media/$USER/ventoy/ --progress -ah

choco install -y ventoy
sudo snap install ventoy
brew cask install ventoy
mount ESP
copy modified esp partition
copy ventoy json


In [ ]:
cd /media/user/SED/VBOX/GitHub/os_from_scratch
#cd .
#sudo virsh define macOS.xml
sudo ln -s ./macOS.xml /etc/libvirt/qemu/macOS.xml
sudo ln -s ./win10-2.xml /etc/libvirt/qemu/win10-2.xml
sudo chown 65543 ESP.qcow2
sudo chown 64055 ESP.qcow2

sdir="/media/$USER/SED/VBOX/GitHub/os_from_scratch/macosx"
tdir="/var/lib/libvirt/images"
cd $sdir
sudo ln -s $sdir/ESP.qcow2 $tdir/ESP.qcow2
sudo ln -s $sdir/CloverNG.qcow2 $tdir/CloverNG.qcow2
sudo ln -s $sdir/CloverHS.qcow2 $tdir/CloverHS.qcow2
sudo ln -s $sdir/CloverM.qcow2 $tdir/CloverM.qcow2

sdir="/media/$USER/SED/VBOX/GitHub/os_from_scratch/macosx"
tdir="/home/$USER/Documents"
sudo cp $sdir/ESP.qcow2 $tdir/ESP.qcow2
sudo cp $sdir/OVMF_VARS-1024x768.fd $tdir/OVMF_VARS-1024x768.fd
sudo cp $sdir/OVMF_CODE.fd $tdir/OVMF_CODE.fd
sudo VBoxManage internalcommands createrawvmdk -filename "/media/user/SED/VBOX/PDC.vmdk" -rawdisk /dev/sdc
#https://www.dark-hamster.com/server/virtualization/use-qcow2-image-virtualbox/
qemu-img convert -f qcow2 ESP.qcow2 -O vdi ESP.vdi




In [ ]:
#Ryujinx Ubuntu
#https://www.youtube.com/watch?v=_NiAEUZBJOk
#Yuzu Ubuntu
#https://www.youtube.com/watch?v=6R8eXwaCthM

#Dep
sudo apt install libsdl2-dev -y
sudo apt install build-essential -y
sudo apt install cmake git -y
sudo apt install libqt5webenginewidgets5 libsndio-dev qtbase5-dev libqt5opengl5-dev -y
sudo ln -T /usr/lib/x86_64-linux-gnu/libsndio.so.7.0 /usr/lib/x86_64-linux-gnu/libsndio.so.6.1

#Run
vblank_mode=0 mesa_glthread=true MESA_GL_VERSION_OVERRIDE=4.5COMPAT  yuzu #AMD & Intel
sudo apt-get install libboost-dev libboost-context-dev libzip-dev -y
__GL_THREADED_OPTIMIZATIONS=1 ./yuzu #NVIDIA

In [ ]:
#w10pd2run
sudo modprobe loop
sudo modprobe linear
dd if=/dev/zero of=efi1 bs=1M count=100
dd if=/dev/zero of=efi2 bs=1M count=1
sudo losetup -f efi1
sudo losetup -f efi2
losetup -a | grep efi1
myFirst=$(losetup -a | grep efi1 | sed -E  's/:/\1/')
umount /dev/nvme1n1 
#sudo mdadm --build --verbose /dev/md0 --chunk=512 --level=linear --raid-devices=3 /dev/loop17 /dev/sda2 /dev/loop18
#sudo mdadm --build --verbose /dev/md0 --chunk=512 --level=linear --raid-devices=3 /dev/loop23 /dev/sda2 /dev/loop24
#grep efi
efi1=$(losetup -a | grep efi1 | sed -r 's/:(.*)/ /')
efi2=$(losetup -a | grep efi2 | sed -r 's/:(.*)/ /')
sudo mdadm --build --verbose /dev/md0 --chunk=512 --level=linear --raid-devices=3 $efi1 /dev/nvme1n1 $efi2
#startwithvirt-manager
virsh start win10-2

sleep 1
sudo chown $USER:$USER /dev/md0


#startwithqemu
sudo qemu-system-x86_64 \
	-bios /usr/share/OVMF/OVMF_CODE.fd \
	-drive file=/dev/md0,media=disk,format=raw \
	-cpu host -enable-kvm -m 8192 \
	-net nic,model=e1000 -net user \
	-smp 4,sockets=1,cores=2,threads=2 \
	-drive file=/home/user/Documents/vm/virtio-win-0.1.185.iso,index=3,media=cdrom
#	-usb -device usb-tablet \
#	-smp cores=4,threads=8 \
#	-rtc base=localtime,clock=host \
#	-net user,smb=$HOME \
#	-device e1000,netdev=net0 \
#	-monitor stdio

In [ ]:
#w10pd2install
SCDIR="$( cd "$( dirname "${BASH_SOURCE[0]}" )" >/dev/null 2>&1 && pwd )"
ISOIMG=${SCDIR}/Win10_2004_EnglishInternational_x64.iso
VIRTIMG=${SCDIR}/virtio-win-0.1.185.iso
HDDIMG=${SCDIR}/win10.img

diskname="nvme-WDC_PC_SN720_SDAQNTW-512G-1001_184356801999" #PopOS
diskname="nvme-SAMSUNG_MZVLB512HAJQ-000L7_S3TNNX0M501502" #W10
diskdev=$(ls -l /dev/disk/by-id | grep "$diskname" | grep -Eo ' -> ../../.*$' | sed -E  's/ -> ..\/..\///' | head -1)
sudo modprobe loop
sudo modprobe linear
dd if=/dev/zero of=efi1 bs=1M count=100
dd if=/dev/zero of=efi2 bs=1M count=1
sudo losetup -f efi1
sudo losetup -f efi2
losetup -a | grep efi1
#myFirst=$(losetup -a | grep efi1 | sed -E  's/:/\1/')
umount /dev/$diskdev
#sudo mdadm --build --verbose /dev/md0 --chunk=512 --level=linear --raid-devices=3 /dev/loop17 /dev/sda2 /dev/loop18
#sudo mdadm --build --verbose /dev/md0 --chunk=512 --level=linear --raid-devices=3 /dev/loop23 /dev/sda2 /dev/loop24
#grep efi

efi1=$(losetup -a | grep efi1 | sed -r 's/:(.*)/ /')
efi2=$(losetup -a | grep efi2 | sed -r 's/:(.*)/ /')
diskdev=$(ls -l /dev/disk/by-id | grep "$diskname" | grep -Eo ' -> ../../.*$' | sed -E  's/ -> ..\/..\///' | head -1)
sudo mdadm --build --verbose /dev/md0 --chunk=512 --level=linear --raid-devices=3 $efi1 /dev/$diskdev $efi2
#umount /dev/sda2
#https://lejenome.tik.tn/post/boot-physical-windows-inside-qemu-guest-machine
#https://bbs.archlinux.org/viewtopic.php?id=233686
# depends on size of efi1 file
# depends on size of efi1 and efi2 files
sudo parted /dev/md0
unit s
mktable gpt
mkpart primary fat32 2048 204799
mkpart primary ntfs 204800 -2049
set 1 boot on
set 1 esp on
set 2 msftdata on
name 1 EFI
name 2 Windows
quit
sudo mkfs.msdos -F 32 -n EFI /dev/md0p1
sudo chown $USER:$USER /dev/md0  # We need to change the owner of md0

sudo qemu-system-x86_64 \
    -bios /usr/share/OVMF/OVMF_CODE.fd \
    -drive file=/dev/sdb,media=disk,format=raw \
    -cpu host -enable-kvm -m 4G \
    -drive file=/dev/md0,media=disk,format=raw \
    -cdrom /home/user/Documents/vm/Win10_2004_EnglishInternational_x64.iso
   
#SHIFT+F10
diskpart
list disk
select disk 0    # Select the disk
list volume      # Find EFI volume (partition) number
select volume 2  # Select EFI volume
assign letter=B  # Assign B: to EFI volume
exit

bcdboot C:\Windows /s B: /f ALL

In [ ]:
#macosrun
# qemu-img create -f qcow2 mac_hdd_ng.img 128G
#
# echo 1 > /sys/module/kvm/parameters/ignore_msrs (this is required)

############################################################################
# NOTE: Tweak the "MY_OPTIONS" line in case you are having booting problems!
############################################################################

# This works for Catalina as well as Mojave. Tested with macOS 10.14.6 and macOS 10.15.

cd /home/user/Documents/vm/MacOS_Catalina/OSX-KVM
MY_OPTIONS="+pcid,+ssse3,+sse4.2,+popcnt,+avx,+aes,+xsave,+xsaveopt,check"

# OVMF=./firmware
OVMF="./backup"

# This causes high cpu usage on the *host* side
# qemu-system-x86_64 -enable-kvm -m 3072 -cpu Penryn,vendor=GenuineIntel,+invtsc,vmware-cpuid-freq=on,hypervisor=off,vmx=on,kvm=off,$MY_OPTIONS\

sudo qemu-system-x86_64 -enable-kvm -m 8192 -cpu Penryn,kvm=on,vendor=GenuineIntel,+invtsc,vmware-cpuid-freq=on,$MY_OPTIONS\
	  -machine q35 \
	  -smp 4,cores=2 \
	  -usb -device usb-kbd -device usb-mouse \
	  -device isa-applesmc,osk="ourhardworkbythesewordsguardedpleasedontsteal(c)AppleComputerInc" \
	  -smbios type=2 \
	  -device ich9-intel-hda -device hda-duplex \
	  -device ich9-ahci,id=sata \
	  -device ide-hd,bus=sata.2,drive=Clover \
	  -device ide-hd,bus=sata.3,drive=InstallMedia \
	  -drive if=pflash,format=raw,readonly,file=$OVMF/OVMF_CODE.fd \
	  -drive if=pflash,format=raw,file=$OVMF/OVMF_VARS-1024x768.fd \
	  -drive id=Clover,if=none,snapshot=on,format=qcow2,file=./'Catalina/CloverNG.qcow2' \
	  -drive id=InstallMedia,if=none,file=BaseSystem.img,format=raw \
	  -drive id=MacHDD,if=none,file=./mac_hdd_ng.img,format=qcow2 \
	  -drive file=/dev/md,media=disk,format=raw \
	  -device ide-hd,bus=sata.4,drive=MacHDD \
	  -netdev tap,id=net0,ifname=tap0,script=no,downscript=no -device vmxnet3,netdev=net0,id=net0,mac=52:54:00:c9:18:27 \
	  -monitor stdio \
	  -vga vmware

In [ ]:
#w10pdrun
sudo qemu-system-x86_64 --enable-kvm -cpu host -m 8192 \
-smp 4,sockets=1,cores=2,threads=2 -mem-path /dev/hugepages \
-vga qxl -display none -serial mon:stdio \
-rtc clock=host,base=localtime \
-device qemu-xhci,id=xhci \
-device virtio-tablet,wheel-axis=true \
-soundhw ac97 \
-netdev user,id=vmnic,smb=/ \
-device virtio-net,netdev=vmnic \
-drive file=/usr/share/OVMF/OVMF_CODE.fd,if=pflash,format=raw,unit=0,readonly=on \
-drive file=/home/user/vm/OVMF_VARS.fd,if=pflash,format=raw,unit=1 \
-drive file=/dev/sda,index=0,media=disk,driver=raw \
-cdrom /home/user/vm/refind-cd-0.12.0.iso

#w10run
SCDIR="$( cd "$( dirname "${BASH_SOURCE[0]}" )" >/dev/null 2>&1 && pwd )"
ISOIMG=${SCDIR}/Win10_2004_EnglishInternational_x64.iso
VIRTIMG=${SCDIR}/virtio-win-0.1.185.iso
HDDIMG=${SCDIR}/W10.img

sudo qemu-system-x86_64 --enable-kvm \
-drive driver=raw,file=${HDDIMG},if=virtio -m 8192 \
-net nic,model=virtio -net user \
-rtc base=localtime,clock=host \
-smp cores=2,threads=4 \
-usb -device usb-tablet \
-net user,smb=$HOME \
-drive file=${VIRTIMG},index=3,media=cdrom \
#-cdrom ${ISOIMG} \
#-drive file=${VIRTIMG},index=3,media=cdrom \
#-net nic,model=virtio -net user \
#-soundhw ac97 \
#-device e1000,netdev=net0 \
#-netdev user,id=net0 \


#w10install
qemu-system-x86_64 --enable-kvm \
-drive driver=raw,file=${HDDIMG},if=virtio -m 6144 \
-device e1000,netdev=net0 \
-netdev user,id=net0 \
-rtc base=localtime,clock=host \
-smp cores=4,threads=8 \
-usb -device usb-tablet \
-net user,smb=$HOME
-cdrom ${ISOIMG} \
-drive file=${VIRTIMG},index=3,media=cdrom \
#-net nic,model=virtio -net user \
#-soundhw ac97 \

In [ ]:
#andx86run
SCDIR="$( cd "$( dirname "${BASH_SOURCE[0]}" )" >/dev/null 2>&1 && pwd )"
ISOIMG=${SCDIR}/android-x86_64-9.0-r2-k49.iso
HDDIMG=${SCDIR}/andx86_64_9.img
set -xe

qemu-img create -f qcow2 "${HDDIMG}" 4G

qemu-system-x86_64 \
	#-enable-kvm -vga std \
	-drive driver=raw \
	-m 2048 -smp 2 -cpu host \
	-soundhw ac97 \
	-net nic,model=e1000 -net user \
	#-cdrom "${ISOIMG}" \
	-hda "${HDDIMG}" \
	-boot d \
	-monitor stdio

In [ ]:
#w10install
qemu-system-x86_64 --enable-kvm \
-drive driver=raw,file=${HDDIMG},if=virtio -m 6144 \
-device e1000,netdev=net0 \
-netdev user,id=net0 \
-rtc base=localtime,clock=host \
-smp cores=4,threads=8 \
-usb -device usb-tablet \
-net user,smb=$HOME
-cdrom ${ISOIMG} \
-drive file=${VIRTIMG},index=3,media=cdrom \
#-net nic,model=virtio -net user \
#-soundhw ac97 \

In [ ]:
#andx86install
SCDIR="$( cd "$( dirname "${BASH_SOURCE[0]}" )" >/dev/null 2>&1 && pwd )"
ISOIMG=${SCDIR}/android-x86_64-9.0-r2-k49.iso
HDDIMG=${SCDIR}/andx86_64_9.img
set -xe

qemu-img create -f qcow2 "${HDDIMG}" 4G

qemu-system-x86_64 \
	#-enable-kvm -vga std \
	-drive driver=raw \
	-m 2048 -smp 2 -cpu host \
	-soundhw ac97 \
	-net nic,model=e1000 -net user \
	-cdrom "${ISOIMG}" \
	-hda "${HDDIMG}" \
	-boot d \
	-monitor stdio

In [ ]:
# MacOSX-kholia template works

#!/usr/bin/env bash

# qemu-img create -f qcow2 mac_hdd_ng.img 128G
#
# echo 1 > /sys/module/kvm/parameters/ignore_msrs (this is required)

############################################################################
# NOTE: Tweak the "MY_OPTIONS" line in case you are having booting problems!
############################################################################

# This works for Catalina as well as Mojave. Tested with macOS 10.14.6 and macOS 10.15.

MY_OPTIONS="+pcid,+ssse3,+sse4.2,+popcnt,+avx,+aes,+xsave,+xsaveopt,check"

# OVMF=./firmware
#OVMF="/home/user/Documents"
OVMF="./backup"

# This causes high cpu usage on the *host* side
# qemu-system-x86_64 -enable-kvm -m 3072 -cpu Penryn,vendor=GenuineIntel,+invtsc,vmware-cpuid-freq=on,hypervisor=off,vmx=on,kvm=off,$MY_OPTIONS\
#-drive id=MacHDD,if=none,file=./mac_hdd_ng.img,format=qcow2 \
#	  -device ide-hd,bus=sata.4,drive=MacHDD \

qemu-system-x86_64 -enable-kvm -m 3072 -cpu Penryn,kvm=on,vendor=GenuineIntel,+invtsc,vmware-cpuid-freq=on,$MY_OPTIONS\
	  -machine q35 \
	  -smp 4,cores=2 \
	  -usb -device usb-kbd -device usb-mouse \
	  -device isa-applesmc,osk="ourhardworkbythesewordsguardedpleasedontsteal(c)AppleComputerInc" \
	  -drive if=pflash,format=raw,readonly,file=$OVMF/OVMF_CODE.fd \
	  -drive if=pflash,format=raw,file=$OVMF/OVMF_VARS-1024x768.fd \
	  -smbios type=2 \
	  -device ich9-intel-hda -device hda-duplex \
	  -device ich9-ahci,id=sata \
	  -drive id=Clover,if=none,snapshot=on,format=qcow2,file=./'Catalina/CloverNG.qcow2' \
	  -device ide-hd,bus=sata.2,drive=Clover \
	  -device ide-hd,bus=sata.3,drive=InstallMedia \
	  -drive id=InstallMedia,if=none,file=BaseSystem.img,format=raw \
	  -drive file=/dev/sdc,media=disk,format=raw \
	  -netdev tap,id=net0,ifname=tap0,script=no,downscript=no -device vmxnet3,netdev=net0,id=net0,mac=52:54:00:c9:18:27 \
	  -monitor stdio \
	  -vga vmware

In [ ]:
# MacOSX-foxlet OVMF_VARS, OVMF_CODE, ESP.qcow2

#!/bin/bash

OSK="ourhardworkbythesewordsguardedpleasedontsteal(c)AppleComputerInc"
VMDIR=$PWD
OVMF=$VMDIR/firmware
#export QEMU_AUDIO_DRV=pa
#QEMU_AUDIO_DRV=pa
#    -drive id=SystemDisk,if=none,file=MyDisk.qcow2 \
#    -device ide-hd,bus=sata.4,drive=SystemDisk \

qemu-system-x86_64 \
    -enable-kvm \
    -m 2G \
    -machine q35,accel=kvm \
    -smp 4,cores=2 \
    -cpu Penryn,vendor=GenuineIntel,kvm=on,+sse3,+sse4.2,+aes,+xsave,+avx,+xsaveopt,+xsavec,+xgetbv1,+avx2,+bmi2,+smep,+bmi1,+fma,+movbe,+invtsc \
    -device isa-applesmc,osk="$OSK" \
    -smbios type=2 \
    -drive if=pflash,format=raw,readonly,file="$OVMF/OVMF_CODE.fd" \
    -drive if=pflash,format=raw,file="$OVMF/OVMF_VARS-1024x768.fd" \
    -vga qxl \
    -device ich9-intel-hda -device hda-output \
    -usb -device usb-kbd -device usb-mouse \
    -netdev user,id=net0 \
    -device e1000-82545em,netdev=net0,id=net0,mac=52:54:00:c9:18:27 \
    -device ich9-ahci,id=sata \
    -drive id=ESP,if=none,format=qcow2,file=ESP.qcow2 \
    -device ide-hd,bus=sata.2,drive=ESP \
    -drive id=InstallMedia,format=raw,if=none,file=BaseSystem.img \
    -device ide-hd,bus=sata.3,drive=InstallMedia \
    #-drive file=/dev/sdc,media=disk,format=raw \